In [139]:
import polars as pl
from src.main import GetTanksReq, fetch_tank_data

req = GetTanksReq(primo_ids={"480001"})

df = await fetch_tank_data(req)
if df is None:
    raise ValueError("df is None")
lf = df.lazy()
lf.collect()

primo_id,scada_id,metric_nice_name,unique_id,uom,timestamp,value,tanksize
str,str,str,object,str,datetime[μs],f64,null
"""480001""","""FTW480001""","""ESD-OilTankInchesUntilAlarm""",0a5456a7-80da-44cd-b191-130a0a579c33,"""in""",2024-03-06 04:14:39.113032,66.518221,null
"""480001""","""FTW480001""","""WaterTank1Capacity""",1a909c77-d910-4862-b7f4-0ae902c9cbab,"""bbl""",2023-11-07 09:00:00,410.476513,null
"""480001""","""FTW480001""","""WaterTank1Volume""",33b18ec5-4680-4279-bc0e-b9f8761d9b04,"""bbl""",2024-03-06 04:11:04.372011,58.378953,null
"""480001""","""FTW480001""","""WaterTank2Volume""",393a9fd5-866d-45d9-98f2-3145da3eab20,"""bbl""",2024-03-06 04:11:21.206990,232.610547,null
"""480001""","""FTW480001""","""OilTank1Level""",3eb86565-9a8c-4645-9e3b-ead0c0a61245,"""in""",2024-03-06 04:12:36.242968,81.727208,null
…,…,…,…,…,…,…,…
"""480001""","""FTW480001""","""ESD-OilTankID""",91346655-25a8-4a44-a3ac-c8fc14457107,null,2023-11-07 09:00:00,1.0,null
"""480001""","""FTW480001""","""OilTank1Volume""",970e57ed-34f9-4306-b0a3-5cd7b57cd20e,"""bbl""",2024-03-06 04:12:36.242968,227.020022,null
"""480001""","""FTW480001""","""OilTank2Capacity""",9e41a2e8-78d5-4dd9-b63f-66fa4676dcfb,"""bbl""",2023-11-07 09:00:00,406.099328,null


In [140]:
lf = lf.drop("unique_id")
lf.collect()

primo_id,scada_id,metric_nice_name,uom,timestamp,value,tanksize
str,str,str,str,datetime[μs],f64,null
"""480001""","""FTW480001""","""ESD-OilTankInchesUntilAlarm""","""in""",2024-03-06 04:14:39.113032,66.518221,null
"""480001""","""FTW480001""","""WaterTank1Capacity""","""bbl""",2023-11-07 09:00:00,410.476513,null
"""480001""","""FTW480001""","""WaterTank1Volume""","""bbl""",2024-03-06 04:11:04.372011,58.378953,null
"""480001""","""FTW480001""","""WaterTank2Volume""","""bbl""",2024-03-06 04:11:21.206990,232.610547,null
"""480001""","""FTW480001""","""OilTank1Level""","""in""",2024-03-06 04:12:36.242968,81.727208,null
…,…,…,…,…,…,…
"""480001""","""FTW480001""","""ESD-OilTankID""",null,2023-11-07 09:00:00,1.0,null
"""480001""","""FTW480001""","""OilTank1Volume""","""bbl""",2024-03-06 04:12:36.242968,227.020022,null
"""480001""","""FTW480001""","""OilTank2Capacity""","""bbl""",2023-11-07 09:00:00,406.099328,null


In [141]:
tank_metrics = ["Level", "Volume", "InchesUntilAlarm", "InchesToESD", "TimeUntilESD", "Capacity", "ID"]
tank_types = ["Water", "Oil"]

tank_metrics_str= "|".join(tank_metrics)
tank_types_str = "|".join(tank_types)

pattern = f'^(?<is_ESD>ESD-)?(?<tank_type>{tank_types_str})Tank(?<tank_number>[0-9]*)(?<tank_metric>{tank_metrics_str})'
lf = lf.with_columns(separated_metrics = pl.col("metric_nice_name").str.extract_groups(pattern))
lf.collect()

primo_id,scada_id,metric_nice_name,uom,timestamp,value,tanksize,separated_metrics
str,str,str,str,datetime[μs],f64,null,struct[4]
"""480001""","""FTW480001""","""ESD-OilTankInchesUntilAlarm""","""in""",2024-03-06 04:14:39.113032,66.518221,null,"{""ESD-"",""Oil"","""",""InchesUntilAlarm""}"
"""480001""","""FTW480001""","""WaterTank1Capacity""","""bbl""",2023-11-07 09:00:00,410.476513,null,"{null,""Water"",""1"",""Capacity""}"
"""480001""","""FTW480001""","""WaterTank1Volume""","""bbl""",2024-03-06 04:11:04.372011,58.378953,null,"{null,""Water"",""1"",""Volume""}"
"""480001""","""FTW480001""","""WaterTank2Volume""","""bbl""",2024-03-06 04:11:21.206990,232.610547,null,"{null,""Water"",""2"",""Volume""}"
"""480001""","""FTW480001""","""OilTank1Level""","""in""",2024-03-06 04:12:36.242968,81.727208,null,"{null,""Oil"",""1"",""Level""}"
…,…,…,…,…,…,…,…
"""480001""","""FTW480001""","""ESD-OilTankID""",null,2023-11-07 09:00:00,1.0,null,"{""ESD-"",""Oil"","""",""ID""}"
"""480001""","""FTW480001""","""OilTank1Volume""","""bbl""",2024-03-06 04:12:36.242968,227.020022,null,"{null,""Oil"",""1"",""Volume""}"
"""480001""","""FTW480001""","""OilTank2Capacity""","""bbl""",2023-11-07 09:00:00,406.099328,null,"{null,""Oil"",""2"",""Capacity""}"


In [142]:
lf = lf.unnest("separated_metrics")
lf.collect()

primo_id,scada_id,metric_nice_name,uom,timestamp,value,tanksize,is_ESD,tank_type,tank_number,tank_metric
str,str,str,str,datetime[μs],f64,null,str,str,str,str
"""480001""","""FTW480001""","""ESD-OilTankInchesUntilAlarm""","""in""",2024-03-06 04:14:39.113032,66.518221,null,"""ESD-""","""Oil""","""""","""InchesUntilAlarm"""
"""480001""","""FTW480001""","""WaterTank1Capacity""","""bbl""",2023-11-07 09:00:00,410.476513,null,null,"""Water""","""1""","""Capacity"""
"""480001""","""FTW480001""","""WaterTank1Volume""","""bbl""",2024-03-06 04:11:04.372011,58.378953,null,null,"""Water""","""1""","""Volume"""
"""480001""","""FTW480001""","""WaterTank2Volume""","""bbl""",2024-03-06 04:11:21.206990,232.610547,null,null,"""Water""","""2""","""Volume"""
"""480001""","""FTW480001""","""OilTank1Level""","""in""",2024-03-06 04:12:36.242968,81.727208,null,null,"""Oil""","""1""","""Level"""
…,…,…,…,…,…,…,…,…,…,…
"""480001""","""FTW480001""","""ESD-OilTankID""",null,2023-11-07 09:00:00,1.0,null,"""ESD-""","""Oil""","""""","""ID"""
"""480001""","""FTW480001""","""OilTank1Volume""","""bbl""",2024-03-06 04:12:36.242968,227.020022,null,null,"""Oil""","""1""","""Volume"""
"""480001""","""FTW480001""","""OilTank2Capacity""","""bbl""",2023-11-07 09:00:00,406.099328,null,null,"""Oil""","""2""","""Capacity"""


In [143]:
lf = lf.with_columns(pl.col("tank_number").cast(pl.UInt8, strict=False))
lf.collect()

primo_id,scada_id,metric_nice_name,uom,timestamp,value,tanksize,is_ESD,tank_type,tank_number,tank_metric
str,str,str,str,datetime[μs],f64,null,str,str,u8,str
"""480001""","""FTW480001""","""ESD-OilTankInchesUntilAlarm""","""in""",2024-03-06 04:14:39.113032,66.518221,null,"""ESD-""","""Oil""",null,"""InchesUntilAlarm"""
"""480001""","""FTW480001""","""WaterTank1Capacity""","""bbl""",2023-11-07 09:00:00,410.476513,null,null,"""Water""",1,"""Capacity"""
"""480001""","""FTW480001""","""WaterTank1Volume""","""bbl""",2024-03-06 04:11:04.372011,58.378953,null,null,"""Water""",1,"""Volume"""
"""480001""","""FTW480001""","""WaterTank2Volume""","""bbl""",2024-03-06 04:11:21.206990,232.610547,null,null,"""Water""",2,"""Volume"""
"""480001""","""FTW480001""","""OilTank1Level""","""in""",2024-03-06 04:12:36.242968,81.727208,null,null,"""Oil""",1,"""Level"""
…,…,…,…,…,…,…,…,…,…,…
"""480001""","""FTW480001""","""ESD-OilTankID""",null,2023-11-07 09:00:00,1.0,null,"""ESD-""","""Oil""",null,"""ID"""
"""480001""","""FTW480001""","""OilTank1Volume""","""bbl""",2024-03-06 04:12:36.242968,227.020022,null,null,"""Oil""",1,"""Volume"""
"""480001""","""FTW480001""","""OilTank2Capacity""","""bbl""",2023-11-07 09:00:00,406.099328,null,null,"""Oil""",2,"""Capacity"""


In [144]:
#pivoting the data
values = pl.col("value")
columns = pl.col("tank_metric")
pivoted_lf = lf.group_by("primo_id", "tank_type", "tank_number", "scada_id").agg(
    values.filter(columns == metric).first().alias(metric) for metric in tank_metrics
)

pivoted_lf.collect()

primo_id,tank_type,tank_number,scada_id,Level,Volume,InchesUntilAlarm,InchesToESD,TimeUntilESD,Capacity,ID
str,str,u8,str,f64,f64,f64,f64,f64,f64,f64
"""480001""","""Oil""",1,"""FTW480001""",81.727208,227.020022,null,null,null,413.344118,null
"""480001""","""Oil""",2,"""FTW480001""",98.825588,274.515522,null,null,null,406.099328,null
"""480001""","""Water""",null,"""FTW480001""",null,null,173.40877,null,null,null,1.0
"""480001""","""Water""",1,"""FTW480001""",21.016423,58.378953,null,null,null,410.476513,null
"""480001""","""Water""",2,"""FTW480001""",83.739797,232.610547,null,null,null,409.718222,null
"""480001""","""Oil""",null,"""FTW480001""",null,null,66.518221,null,null,null,1.0


In [145]:
null_condition = pl.col("tank_number").is_null()
not_null_condition = pl.col("tank_number").is_not_null()

null_tanks = pivoted_lf.filter(null_condition)
numbered_tanks = pivoted_lf.filter(not_null_condition)

null_tanks = null_tanks.with_columns(pl.col("ID").alias("tank_number"))
null_tanks = null_tanks.drop("ID")
null_tanks = null_tanks.with_columns(pl.col("tank_number").cast(pl.UInt8, strict=False))

null_tanks.collect()

primo_id,tank_type,tank_number,scada_id,Level,Volume,InchesUntilAlarm,InchesToESD,TimeUntilESD,Capacity
str,str,u8,str,f64,f64,f64,f64,f64,f64
"""480001""","""Water""",1,"""FTW480001""",null,null,173.40877,null,null,null
"""480001""","""Oil""",1,"""FTW480001""",null,null,66.518221,null,null,null


In [146]:
joined_lf = numbered_tanks.join(null_tanks, on=["primo_id", "tank_type", "tank_number"], how="left")
joined_lf.collect()

primo_id,tank_type,tank_number,scada_id,Level,Volume,InchesUntilAlarm,InchesToESD,TimeUntilESD,Capacity,ID,scada_id_right,Level_right,Volume_right,InchesUntilAlarm_right,InchesToESD_right,TimeUntilESD_right,Capacity_right
str,str,u8,str,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64
"""480001""","""Water""",1,"""FTW480001""",21.016423,58.378953,null,null,null,410.476513,null,"""FTW480001""",null,null,173.40877,null,null,null
"""480001""","""Oil""",1,"""FTW480001""",81.727208,227.020022,null,null,null,413.344118,null,"""FTW480001""",null,null,66.518221,null,null,null
"""480001""","""Water""",2,"""FTW480001""",83.739797,232.610547,null,null,null,409.718222,null,null,null,null,null,null,null,null
"""480001""","""Oil""",2,"""FTW480001""",98.825588,274.515522,null,null,null,406.099328,null,null,null,null,null,null,null,null


In [147]:
final_lf = joined_lf.join(lf, on=["scada_id"], how="left")
final_lf = final_lf.group_by("primo_id", "tank_type", "tank_number").agg(pl.all().first())
final_lf.collect()

primo_id,tank_type,tank_number,scada_id,Level,Volume,InchesUntilAlarm,InchesToESD,TimeUntilESD,Capacity,ID,scada_id_right,Level_right,Volume_right,InchesUntilAlarm_right,InchesToESD_right,TimeUntilESD_right,Capacity_right,primo_id_right,metric_nice_name,uom,timestamp,value,tanksize,is_ESD,tank_type_right,tank_number_right,tank_metric
str,str,u8,str,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,str,str,str,datetime[μs],f64,null,str,str,u8,str
"""480001""","""Water""",2,"""FTW480001""",83.739797,232.610547,null,null,null,409.718222,null,null,null,null,null,null,null,null,"""480001""","""ESD-OilTankInchesUntilAlarm""","""in""",2024-03-06 04:14:39.113032,66.518221,null,"""ESD-""","""Oil""",null,"""InchesUntilAlarm"""
"""480001""","""Water""",1,"""FTW480001""",21.016423,58.378953,null,null,null,410.476513,null,"""FTW480001""",null,null,173.40877,null,null,null,"""480001""","""ESD-OilTankInchesUntilAlarm""","""in""",2024-03-06 04:14:39.113032,66.518221,null,"""ESD-""","""Oil""",null,"""InchesUntilAlarm"""
"""480001""","""Oil""",2,"""FTW480001""",98.825588,274.515522,null,null,null,406.099328,null,null,null,null,null,null,null,null,"""480001""","""ESD-OilTankInchesUntilAlarm""","""in""",2024-03-06 04:14:39.113032,66.518221,null,"""ESD-""","""Oil""",null,"""InchesUntilAlarm"""
"""480001""","""Oil""",1,"""FTW480001""",81.727208,227.020022,null,null,null,413.344118,null,"""FTW480001""",null,null,66.518221,null,null,null,"""480001""","""ESD-OilTankInchesUntilAlarm""","""in""",2024-03-06 04:14:39.113032,66.518221,null,"""ESD-""","""Oil""",null,"""InchesUntilAlarm"""


In [148]:
final_lf = final_lf.with_columns(
    pl.col("primo_id"),
    pl.col("tank_type"),
    pl.col("tank_number"),
    pl.coalesce(pl.col("Level"), pl.col("Level_right")).alias("Level"),
    pl.coalesce(pl.col("Volume"), pl.col("Volume_right")).alias("Volume"),
    pl.coalesce(pl.col("InchesToESD"), pl.col("InchesUntilAlarm_right")).alias("InchesToESD"),
    pl.coalesce(pl.col("TimeUntilESD"), pl.col("TimeUntilESD_right")).alias("TimeUntilESD"),
    pl.coalesce(pl.col("Capacity"), pl.col("tanksize")).alias("Capacity")
)

required_columns = ["primo_id", "tank_type", "tank_number", "Level", "Volume", "InchesToESD", "TimeUntilESD", "Capacity"]
final_lf = final_lf.select(required_columns)
final_lf.collect()

primo_id,tank_type,tank_number,Level,Volume,InchesToESD,TimeUntilESD,Capacity
str,str,u8,f64,f64,f64,f64,f64
"""480001""","""Oil""",1,81.727208,227.020022,66.518221,null,413.344118
"""480001""","""Oil""",2,98.825588,274.515522,null,null,406.099328
"""480001""","""Water""",1,21.016423,58.378953,173.40877,null,410.476513
"""480001""","""Water""",2,83.739797,232.610547,null,null,409.718222


In [149]:
final_lf = final_lf.sort("primo_id", "tank_type", "tank_number")
final_lf.collect()

primo_id,tank_type,tank_number,Level,Volume,InchesToESD,TimeUntilESD,Capacity
str,str,u8,f64,f64,f64,f64,f64
"""480001""","""Oil""",1,81.727208,227.020022,66.518221,null,413.344118
"""480001""","""Oil""",2,98.825588,274.515522,null,null,406.099328
"""480001""","""Water""",1,21.016423,58.378953,173.40877,null,410.476513
"""480001""","""Water""",2,83.739797,232.610547,null,null,409.718222


In [150]:
percent_tank_full = (pl.col("Volume") / pl.col("Capacity") * 100).round().cast(pl.UInt8)
final_lf = final_lf.with_columns(percent_tank_full.alias("percent_full"))

final_lf.collect()

primo_id,tank_type,tank_number,Level,Volume,InchesToESD,TimeUntilESD,Capacity,percent_full
str,str,u8,f64,f64,f64,f64,f64,u8
"""480001""","""Oil""",1,81.727208,227.020022,66.518221,null,413.344118,55
"""480001""","""Oil""",2,98.825588,274.515522,null,null,406.099328,68
"""480001""","""Water""",1,21.016423,58.378953,173.40877,null,410.476513,14
"""480001""","""Water""",2,83.739797,232.610547,null,null,409.718222,57


In [151]:
inches_to_feet = (pl.col("Level") / 12).round(1)
tank_level_to_str = inches_to_feet.cast(pl.Utf8)

split_col = tank_level_to_str.str.split('.')

tank_level_split_feet = split_col.list.get(0)
tank_level_split_inches = split_col.list.get(1)

tank_level_uom = pl.concat_str([tank_level_split_feet, pl.lit("'"), tank_level_split_inches, pl.lit('"')])

final_lf = final_lf.with_columns(tank_level_uom.alias("tank_level_str"))

final_lf.collect()

primo_id,tank_type,tank_number,Level,Volume,InchesToESD,TimeUntilESD,Capacity,percent_full,tank_level_str
str,str,u8,f64,f64,f64,f64,f64,u8,str
"""480001""","""Oil""",1,81.727208,227.020022,66.518221,null,413.344118,55,"""6'8"""""
"""480001""","""Oil""",2,98.825588,274.515522,null,null,406.099328,68,"""8'2"""""
"""480001""","""Water""",1,21.016423,58.378953,173.40877,null,410.476513,14,"""1'8"""""
"""480001""","""Water""",2,83.739797,232.610547,null,null,409.718222,57,"""7'0"""""


In [152]:
inches_to_feet_esd = (pl.col("InchesToESD") / 12).round(1)

inchesESD_to_feet = inches_to_feet_esd.cast(pl.Utf8)

split_esd = inchesESD_to_feet.str.split('.')

esd_split_feet = split_esd.list.get(0)
esd_split_inches = split_esd.list.get(1)

esd_final = pl.concat_str([esd_split_feet, pl.lit("'"), esd_split_inches, pl.lit('"')])

final_lf = final_lf.with_columns(esd_final.alias("to_ESD"))

final_lf.collect()

primo_id,tank_type,tank_number,Level,Volume,InchesToESD,TimeUntilESD,Capacity,percent_full,tank_level_str,to_ESD
str,str,u8,f64,f64,f64,f64,f64,u8,str,str
"""480001""","""Oil""",1,81.727208,227.020022,66.518221,null,413.344118,55,"""6'8""""","""5'5"""""
"""480001""","""Oil""",2,98.825588,274.515522,null,null,406.099328,68,"""8'2""""",null
"""480001""","""Water""",1,21.016423,58.378953,173.40877,null,410.476513,14,"""1'8""""","""14'5"""""
"""480001""","""Water""",2,83.739797,232.610547,null,null,409.718222,57,"""7'0""""",null
